In [ ]:
import numpy as np
import pandas as pd

In [65]:
# Configuration to automatically reload modified modules
%load_ext autoreload
%autoreload 2

# This allows changes in imported modules to be reflected automatically
# without needing to restart the kernel

# Load Gold Standard Corpus

In [9]:
gold_corpus = pd.read_csv("../Task_3/data/corpus_raw.csv")

# Load Vector Space Model

In [26]:
import os
import sys


# Add the Task_3 directory to the Python path
sys.path.append(os.path.join(os.getcwd(), "../Task_3"))
from vectorizing import load_vectors_scipy

## Load TF (BoW) vectors

In [29]:
tf_vectors, tf_vocab = load_vectors_scipy("../Task_3/data/VSM/tf_vectors")

### K-Means clustering

In [64]:
from clustering import kmeans_pipeline

In [66]:
cluster_labels = kmeans_pipeline(tf_vectors)

/Users/thedandy/Desktop/Master_IA_codigos/text-mining/.venv/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [67]:
indices_valor_3 = np.where(cluster_labels == 1)[0]
gold_corpus.iloc[indices_valor_3]["category"].value_counts()

category
talk.politics.guns          20
talk.politics.mideast       16
rec.sport.hockey             2
comp.sys.ibm.pc.hardware     2
rec.autos                    1
comp.sys.mac.hardware        1
sci.electronics              1
Name: count, dtype: int64

### --- clustering

## Load TF-IDF vectors

In [30]:
tfidf_vectors, tfidf_vocab = load_vectors_scipy("../Task_3/data/VSM/tfidf_vectors")

### K-Means clustering

### --- clustering

# Load Semantic Space Model

## Load Additive Embeddings

## Load Average Embeddings